In [1]:
# import required libraries
import pandas as pd
import string
from nltk.corpus import stopwords

In [2]:
#get the spam data collection using pandas
df_spamcollection = pd.read_csv('Desktop\\SMSSpamCollection.tsv',sep='\t',names=['response','message'])

In [3]:
#view first 5 records
df_spamcollection.head()

,response,message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [4]:
#view more information about the spam data using describe method
df_spamcollection.describe()

,response,message
count,5568,5568
unique,2,5165
top,ham,"Sorry, I'll call later"
freq,4822,30


In [5]:
#view response using group by and describe method
df_spamcollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4822   4513                             Sorry, I'll call later   
spam         746    652  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [6]:
#verify length of the messages and also add it also as a new column (feature)
df_spamcollection['length'] = df_spamcollection['message'].apply(len)

In [7]:
#view first 5 messages with length
df_spamcollection.head()

,response,message,length
0,ham,I've been searching for the right words to tha...,196
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,ham,"Nah I don't think he goes to usf, he lives aro...",61
3,ham,Even my brother is not like to speak with me. ...,77
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,35


In [8]:
#define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    #check characters to see if there are punctuations
    no_punctuation =  [char for char in mess if char not in string.punctuation]
    #now for the sentence
    no_punctuation = ''.join(no_punctuation)
    #now eliminate any stopwords
    return[word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [9]:
#verify that function is working
df_spamcollection['message'].head(5).apply(message_text_process)

0    [Ive, searching, right, words, thank, breather...
1    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
2    [Nah, dont, think, goes, usf, lives, around, t...
3    [Even, brother, like, speak, treat, like, aids...
4                                       [DATE, SUNDAY]
Name: message, dtype: object

In [10]:
#start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
#bag of words by applying the function and fit the data (message)into it
bag_of_words_transformer = CountVectorizer(analyzer=message_text_process).fit(df_spamcollection['message'])

In [13]:
#print lenth of bag of words stored int the vocabulary_ attributes
print (len(bag_of_words_transformer.vocabulary_))

11419


In [14]:
#store bag of words for messages using transform method
message_bagofwords = bag_of_words_transformer.transform(df_spamcollection['message'])

In [15]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords)

In [20]:
#print shape of the tfidf
message_tfidf = tfidf_transformer.transform(message_bagofwords)
print(message_tfidf.shape)

(5568, 11419)


In [23]:
#choose naive bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfidf,df_spamcollection['response'])


In [24]:
#check model for the pedicted and expected value say for message#2 and messafe#5
message = df_spamcollection['message'][2]
bag_of_words_for_message = bag_of_words_transformer.transform([message])
tfidf = tfidf_transformer.transform(bag_of_words_for_message)

print ('predicted',spam_detect_model.predict(tfidf)[0])
print('expected',df_spamcollection.response[4])

predicted ham
expected ham
